## PGx Cohort Pipeline (Driver Notebook)

This notebook serves as the **top-level driver** for running the PGx analysis pipeline for a given `(cohort_name, age_band)` pair.

### High-Level Phases

1. **Data Ingestion & Cleaning** (`1_apcd_input_data/`) – raw TXT → parquet, global imputation, QA (detailed cells below in this notebook).
2. **Cohort Construction** (`2_create_cohort/`) – builds `cohorts_F1120` from medical + pharmacy.
3. **Feature Importance** (`3_feature_importance/`) – Monte Carlo CV feature importance with **CatBoost, XGBoost, XGBoost RF**.
4. **FP-Growth** (`4_fpgrowth_analysis/`) – frequent pattern mining and drug encoding.
5. **Process Mining (bupaR)** (`5_bupaR_analysis/`) – event logs, time-to-event, and time-windowed patterns.
6. **DTW Trajectories** (`6_dtw_analysis/`) – trajectory prototypes and DTW distances.
7. **Final Model Ensemble** (`7_final_model/`) – final feature table + CatBoost / XGBoost / XGBoost RF ensemble.
8. **Formal Feature Analysis** (`8_ffa_analysis/`) – calibration, symbolic rules, and causal analysis.

### Notebook Integration

For deeper analysis in each phase, open the cohort-specific notebooks:

- **Feature Importance (EC2 driver)**: `3_feature_importance/feature_importance_cohort_runner.ipynb`
- **bupaR – opioid_ed**: `5_bupaR_analysis/bupaR_pipeline_opioid_ed.ipynb`
- **bupaR – non_opioid_ed**: `5_bupaR_analysis/bupaR_pipeline_non_opioid_ed.ipynb`
- **DTW – opioid_ed**: `6_dtw_analysis/dtw_pipeline_opioid_ed.ipynb`
- **DTW – non_opioid_ed**: `6_dtw_analysis/dtw_pipeline_non_opioid_ed.ipynb`
- **Final Model Ensemble**: `7_final_model/final_model.ipynb`
- **Formal Feature Analysis (FFA)**: `8_ffa_analysis/catboost_feature_attribution_analysis.ipynb`

The sections below retain the **detailed ingestion, QA, FP-Growth, CatBoost, and FFA code**. Use this notebook as:

- A **driver** (top cells) to orchestrate key phases, and
- A **reference** (lower cells) for full data pipeline details and one-off investigations.

### Final Analysis Workflow (from `README_analysis_workflow.md`)

For the **final analysis**, we organize the downstream methods into three phases:

1. **Phase 1 – Monte Carlo CV + Feature Importance** (`3_feature_importance/`)
   - Three core models: **CatBoost, XGBoost (boosted trees), XGBoost RF mode**.
   - Monte Carlo CV on temporally separated train/test (2016–2018 → 2019) with permutation-based importance.
   - Outputs: ranked feature lists and filtered `model_data` event tables for target (`opioid_ed`) and control (`non_opioid_ed`) cohorts.

2. **Phase 2 – Pattern & Process Mining + DTW** (`4_fpgrowth_analysis/`, `5_bupaR_analysis/`, `6_dtw_analysis/`)
   - **FP-Growth**: frequent itemsets and target-focused rules on selected codes.
   - **bupaR**: event logs, pre/post-target process maps, time-windowed and time-to-event features saved under `5_bupaR_analysis/outputs/.../features/`.
   - **DTW**: cohort- and age-band–specific trajectory features (distances to prototypes) under `6_dtw_analysis/outputs/.../features/`.

3. **Phase 3 – Final Model Development & Attribution** (`7_final_model/`, `8_ffa_analysis/`)
   - Integrate feature-importance–filtered `model_data`, FP-Growth, bupaR, and DTW features into a single patient-level table following `final_feature_schema.json`.
   - Train the final three-model ensemble (CatBoost, XGBoost, XGBoost RF) with temporal validation.
   - Run FFA/attribution analysis to export symbolic rules, SHAP-style summaries, and causal-effect diagnostics.

See `README_analysis_workflow.md` for full details and the end-to-end Mermaid diagram of these phases.

In [ ]:
from pathlib import Path
import os
import sys


def resolve_project_root() -> Path:
    """Resolve the pgx-analysis project root for both notebook and script modes."""
    # When run as a script, __file__ is defined
    if "__file__" in globals():
        return Path(__file__).resolve().parents[0]

    # When run inside Jupyter, fall back to current working directory and parents
    cwd = Path(os.getcwd()).resolve()
    # If we're already in the repo root, keep it
    if (cwd / "pgx_cohort_pipeline.ipynb").exists():
        return cwd

    for parent in cwd.parents:
        if (parent / "pgx_cohort_pipeline.ipynb").exists():
            return parent

    return cwd


PROJECT_ROOT = resolve_project_root()
print(f"[INFO] Project root: {PROJECT_ROOT}")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# ------------------------------------------------------------------
# Cohort configuration for downstream phases
# ------------------------------------------------------------------
COHORT_NAME = "opioid_ed"       # or "non_opioid_ed"
AGE_BAND = "25-44"              # e.g., "0-12", "13-24", ..., "85-94"
TRAIN_YEARS = [2016, 2017, 2018]
TEST_YEAR = 2019

print(
    f"[CONFIG] cohort={COHORT_NAME}, age_band={AGE_BAND}, "
    f"train_years={TRAIN_YEARS}, test_year={TEST_YEAR}"
)


In [ ]:
%%bash

# Delete all partitions to start fresh
aws s3 rm s3://pgxdatalake/bronze/ --recursive

In [ ]:
%%bash

# Delete all partitions to start fresh
aws s3 rm s3://pgxdatalake/silver/ --recursive

In [ ]:
%%bash

# Delete all partitions to start fresh
aws s3 rm s3://pgxdatalake/gold/pharmacy/ --recursive
aws s3 rm s3://pgxdatalake/gold/medical/ --recursive

## A. Input Datasets - Initial Clean

### 1. Text to Parquet Format

- run this from command line on EC2:  
sudo mkdir -p /mnt/nvme/duckdb_tmp  
sudo chown -R "$USER":"$USER" /mnt/nvme/duckdb_tmp  

In [ ]:
%%bash
set -euo pipefail

echo "🚀 TXT → Parquet (bronze) starting..."
echo "Started at: $(date)"
echo ""

mkdir -p /home/pgx3874/pgx-analysis/1_apcd_input_data/logs

/home/pgx3874/jupyter-env/bin/python3.11 /home/pgx3874/pgx-analysis/1_apcd_input_data/0_txt_to_parquet.py \
  --dataset both \
  --workers 18 \
  --duckdb-threads 1 \
  --split-rejects \
  --bronze-root s3://pgxdatalake/bronze/ \
  --overwrite \
  --tmp-dir /mnt/nvme/duckdb_tmp 2>&1 | tee "/home/pgx3874/pgx-analysis/1_apcd_input_data/logs/0_txt_to_parquet_$(date +%Y%m%d_%H%M%S).log"

echo ""
echo "✅ TXT → Parquet (bronze) completed at: $(date)"


In [ ]:
%%bash
set -euo pipefail

echo "🚀 Reprocessing corrected rejects..."
echo "Started at: $(date)"
echo ""

mkdir -p /home/pgx3874/pgx-analysis/1_apcd_input_data/logs

/home/pgx3874/jupyter-env/bin/python3.11 /home/pgx3874/pgx-analysis/1_apcd_input_data/1_reprocess_txt_to_parquet.py \
  --dataset both \
  --workers 18 \
  --duckdb-threads 1 \
  --bronze-root s3://pgxdatalake/bronze/ \
  --tmp-dir /mnt/nvme/duckdb_tmp 2>&1 | tee "/home/pgx3874/pgx-analysis/1_apcd_input_data/logs/1_reprocess_txt_to_parquet_$(date +%Y%m%d_%H%M%S).log"

echo ""
echo "✅ Reprocess completed at: $(date)"

### 2. Clean Pharmacy

In [ ]:
import duckdb

# Enable S3 and HTTPFS support
duckdb.sql("INSTALL httpfs; LOAD httpfs;")
duckdb.sql("CALL load_aws_credentials();")

# Define your input path
pharmacy_input_path = 's3://pgxdatalake/bronze/pharmacy/**/*.parquet'

# Check the schema (grab 0 rows to inspect column names and types only)
schema_df = duckdb.sql(f"""
    DESCRIBE SELECT * FROM read_parquet('{pharmacy_input_path}') LIMIT 0
""").df()

print(schema_df)

#### a. Drug Lookup Map

In [ ]:
! aws s3 sync s3://pgx-repository/drug_mappings/ /home/pgx3874/pgx-analysis/1_apcd_input_data/drug_mappings

In [ ]:
import duckdb

mapping_dir = "/home/pgx3874/pgx-analysis/1_apcd_input_data/drug_mappings"

# Access struct fields from MAP_ENTRIES
mapping_files_query = f"""
SELECT 
  LOWER(key.key) AS key,
  LOWER(key.value) AS value
FROM read_json_auto('{mapping_dir}/*_mappings.json'),
UNNEST(MAP_ENTRIES(json)) AS kv(key)
"""

drug_map = duckdb.sql(mapping_files_query)
drug_map.create("drug_map")

#### b. Impute Missing Data / Create Silver Tables

In [ ]:
%%bash
set -euo pipefail

mkdir -p /home/pgx3874/pgx-analysis/1_apcd_input_data/logs

# Phase 1: Global Imputation (Optimized - No Demographics Lookup)
echo "🚀 Phase 1: Starting Global Demographic Imputation..."
echo "Input: Bronze tier pharmacy and medical data"
echo "Output: Imputed partitioned data"
echo "Started at: $(date)"
echo ""

/home/pgx3874/jupyter-env/bin/python3.11 \
  /home/pgx3874/pgx-analysis/1_apcd_input_data/2_global_imputation.py \
  --pharmacy-input s3://pgxdatalake/bronze/pharmacy/*.parquet \
  --medical-input s3://pgxdatalake/bronze/medical/*.parquet \
  --output-root s3://pgxdatalake/silver/imputed \
  --create-raw-silver \
  --lookahead-years 5 \
  --no-demographics-lookup \
  --log-level INFO 2>&1 | tee "/home/pgx3874/pgx-analysis/1_apcd_input_data/logs/2_global_imputation_$(date +%Y%m%d_%H%M%S).log"

echo ""
echo "✅ Global Imputation completed successfully at: $(date)"

🚀 Phase 1: Starting Global Demographic Imputation...
Input: Bronze tier pharmacy and medical data
Output: Imputed partitioned data
Started at: Wed Nov 26 14:56:51 UTC 2025

2025-11-26 14:56:55,702 - INFO - 🚀 Starting Global Demographic Imputation
2025-11-26 14:56:55,702 - INFO - 📊 Pharmacy input: s3://pgxdatalake/bronze/pharmacy/*.parquet
2025-11-26 14:56:55,702 - INFO - 📊 Medical input: s3://pgxdatalake/bronze/medical/*.parquet
2025-11-26 14:56:55,702 - INFO - 📊 Output root: s3://pgxdatalake/silver/imputed
2025-11-26 14:56:55,702 - INFO - 📊 Lookahead years: 5
2025-11-26 14:56:55,702 - INFO - 📊 Create raw silver: True
2025-11-26 14:56:55,702 - INFO - 📊 DuckDB will auto-detect optimal memory and thread settings
2025-11-26 14:56:55,702 - INFO - 🚀 Starting Global Demographic Imputation
2025-11-26 14:56:55,702 - INFO - 🔧 Using Version 1997 + 12 - Global Imputation (DuckDB Lessons Learned Applied)
2025-11-26 14:56:55,702 - INFO - =============================================================

#### c. Normalize Drug Names

In [ ]:
%%bash
set -euo pipefail

# Phase 2: Optimized Partition Processing using Pre-Imputed Data
echo "🚀 Phase 2: Running Optimized Partition Processing with Pre-Imputed Data..."
echo "Input: Silver tier imputed partitioned data (no demographics lookup needed)"
echo "📁 Output: Gold tier final partitions"
echo " Started at: $(date)"
echo ""

# Create logs directory
mkdir -p /home/pgx3874/pgx-analysis/1_apcd_input_data/logs

job="pharmacy"
PHARMACY_SCRIPT="/home/pgx3874/pgx-analysis/1_apcd_input_data/3a_clean_pharmacy.py"

# Use the imputed partitioned data directly (no demographics lookup needed)
/home/pgx3874/jupyter-env/bin/python3.11 /home/pgx3874/pgx-analysis/1_apcd_input_data/3_apcd_clean.py \
  --job "$job" \
  --pharmacy-input s3://pgxdatalake/silver/imputed/pharmacy_partitioned/**/*.parquet \
  --output-root s3://pgxdatalake/gold/pharmacy \
  --min-year 2016 --max-year 2020 \
  --workers 48 \
  --retries 1 \
  --run-mode subprocess \
  --pharmacy-script "$PHARMACY_SCRIPT" \
  --log-level INFO 2>&1 | tee "/home/pgx3874/pgx-analysis/1_apcd_input_data/logs/${job}_clean_output_$(date +%Y%m%d_%H%M%S).log"

echo ""
echo "✅ Optimized partition processing completed at: $(date)"

### 3. Drug Frequency Analysis

In [ ]:
%%bash

/home/pgx3874/jupyter-env/bin/python3.11 \
   /home/pgx3874/pgx-analysis/1_apcd_input_data/4_drug_frequency_analysis.py

In [ ]:
import pickle

# Load data from script
pickle_path = '/home/pgx3874/pgx-analysis/1_apcd_input_data/outputs/drug_analysis_data.pkl'
with open(pickle_path, 'rb') as f:
    drug_data = pickle.load(f)

print("✅ Data loaded successfully!")

drug_df = drug_data['df']
drug_high_freq_df = drug_data['high_freq_df']
drug_low_freq_df = drug_data['low_freq_df']
drug_summary_df = drug_data['summary_df']
drug_trends_df = drug_data['trends_df']


print(f"📊 Main data: {len(drug_df):,} records")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Reusable visuals
from helpers_1997_13.visualization_utils import (
    plot_stacked_by_year,
    plot_top_bars,
    save_current_chart,
    create_plotly_frequency_dashboard
)


# High frequency drugs stacked by year using reusable function
ordered_high = drug_high_freq_df['drug_name'].tolist()
plot_stacked_by_year(
    drug_df[drug_df['drug_name'].isin(ordered_high)],
    target_col='drug_name',
    year_col='event_year',
    freq_col='frequency',
    ordered_targets=ordered_high,
    title_suffix='High Frequency (>=1000)'
)

save_current_chart('high_freq_by_year_drug', 'drug_frequency')


In [ ]:
# Low frequency drugs stacked by year (limit to top 25 for readability)
ordered_low = drug_low_freq_df.head(25)['drug_name'].tolist()
plot_stacked_by_year(
    drug_df[drug_df['drug_name'].isin(ordered_low)],
    target_col='drug_name',
    year_col='event_year',
    freq_col='frequency',
    ordered_targets=ordered_low,
    title_suffix='Low Frequency (<1000) top25'
)

save_current_chart('low_freq_by_year_drug', 'drug_frequency')

In [ ]:
# Top-N bars using reusable function
plot_top_bars(drug_high_freq_df, target_col='drug_name', value_col='total_frequency', top_n=10, title='Top 10 High Frequency Drugs')
save_current_chart('top10_high_drug', 'drug_frequency')

plot_top_bars(drug_low_freq_df, target_col='drug_name', value_col='total_frequency', top_n=10, title='Top 10 Low Frequency Drugs')
save_current_chart('top10_low_drug', 'drug_frequency')

In [ ]:
# Drug Dashboard
create_plotly_frequency_dashboard(
    drug_df,
    title='Drug Frequency Explorer',
    s3_output_path='s3://pgxdatalake/visualizations/drug_name/drug_frequency_dashboard.html',
    target_col='drug_name', year_col='event_year', freq_col='frequency', system_col=None, top_n=999999
)

### 4. Clean Medical

#### a. Clean Dataset

In [ ]:
%%bash
set -euo pipefail

# Phase 2b: Optimized Medical Processing
echo "🚀 Phase 2b: Starting Optimized Medical Processing..."
echo "Input: Silver tier imputed partitioned medical data"
echo "Output: Gold tier final medical partitions"
echo "Started at: $(date)"
echo ""

# Create logs directory
mkdir -p logs

/home/pgx3874/jupyter-env/bin/python3.11 \
    /home/pgx3874/pgx-analysis/1_apcd_input_data/3_apcd_clean.py \
    --job medical \
    --raw-medical s3://pgxdatalake/silver/medical/*.parquet \
    --output-root s3://pgxdatalake/gold/medical \
    --min-year 2016 --max-year 2020 \
    --workers 9 \
    --retries 1 \
    --run-mode subprocess \
    --medical-script /home/pgx3874/pgx-analysis/1_apcd_input_data/3b_clean_medical.py \
    --log-level INFO 2>&1 | tee "logs/medical_clean_output_$(date +%Y%m%d_%H%M%S).log"

echo ""
echo "✅ Phase 2b completed successfully at: $(date)"

### 5. QA

In [ ]:
%%bash

# Run QA validation on both datasets
/home/pgx3874/jupyter-env/bin/python3.11 \
  1_apcd_input_data/5_step1_data_quality_qa.py \
  --type both \
  --all-partitions \
  --workers 16 \
  --save-results \
  --verbose \
  2>&1 | tee 1_apcd_input_data/logs/qa_results_$(date +%Y%m%d_%H%M%S).log

### 6. Glue Crawler for Athena

In [ ]:
import boto3
import time
from botocore.exceptions import ClientError

# --- Configuration ---
CRAWLER_NAME = "medical"
REGION = "us-east-1"  # Change this if your crawler is in another region

# --- Initialize the Glue client ---
glue = boto3.client("glue", region_name=REGION)

# --- Start the crawler (safely) ---
try:
    crawler = glue.get_crawler(Name=CRAWLER_NAME)
    state = crawler["Crawler"]["State"]

    if state == "READY":
        print(f"Starting crawler: {CRAWLER_NAME} ...")
        glue.start_crawler(Name=CRAWLER_NAME)
    else:
        print(f"Crawler '{CRAWLER_NAME}' is currently {state}. Waiting...")

except ClientError as e:
    if e.response["Error"]["Code"] == "CrawlerRunningException":
        print(f"Crawler '{CRAWLER_NAME}' is already running.")
    else:
        raise

# --- Monitor progress until completion ---
while True:
    crawler = glue.get_crawler(Name=CRAWLER_NAME)
    state = crawler["Crawler"]["State"]
    last_run = crawler["Crawler"].get("LastCrawl", {}).get("Status")

    print(f"[{time.strftime('%X')}] Crawler state: {state} | Last run: {last_run}")
    if state == "READY":
        print(f"Crawler '{CRAWLER_NAME}' has completed.")
        break
    time.sleep(30)



In [ ]:

CRAWLER_NAME2 = "pharmacy"

try:
    crawler = glue.get_crawler(Name=CRAWLER_NAME2)
    state = crawler["Crawler"]["State"]

    if state == "READY":
        print(f"Starting crawler: {CRAWLER_NAME2} ...")
        glue.start_crawler(Name=CRAWLER_NAME2)
    else:
        print(f"Crawler '{CRAWLER_NAME2}' is currently {state}. Waiting...")

except ClientError as e:
    if e.response["Error"]["Code"] == "CrawlerRunningException":
        print(f"Crawler '{CRAWLER_NAME2}' is already running.")
    else:
        raise

while True:
    crawler = glue.get_crawler(Name=CRAWLER_NAME2)
    state = crawler["Crawler"]["State"]
    last_run = crawler["Crawler"].get("LastCrawl", {}).get("Status")

    print(f"[{time.strftime('%X')}] Crawler state: {state} | Last run: {last_run}")
    if state == "READY":
        print(f"Crawler '{CRAWLER_NAME2}' has completed.")
        break
    time.sleep(30)


### 7. Target Variable

#### a. Target Variable Frequency Analysis

In [ ]:
%%bash

export PGX_WORKERS_MEDICAL=16
export PGX_THREADS_PER_WORKER=1
export PGX_S3_MAX_CONNECTIONS=64  

/home/pgx3874/jupyter-env/bin/python3.11 \
  /home/pgx3874/pgx-analysis/1_apcd_input_data/6_target_frequency_analysis.py \
  --codes-of-interest "F11.20" \
  --workers ${PGX_WORKERS_MEDICAL} \
  --min-year 2016 --max-year 2020 \
  --log-cpu --log-s3

#### b. Update Target Codes

In [ ]:
%%bash

set -euo pipefail

# ========================================
# LOCAL STAGING: Maximum Performance
# ========================================
export PGX_USE_LOCAL_STAGING=1  # Enable local staging (default: on)
export PGX_LOCAL_STAGING_DIR="/mnt/nvme/duckdb_tmp"  # Use fast NVMe

# ========================================
# WORKER CONFIGURATION: 24 workers
# ========================================
export PGX_WORKERS_MEDICAL=28  # High parallelism (safe with local staging)
export PGX_THREADS_PER_WORKER=1
export PGX_TOTAL_WORKERS=28
export PGX_DUCKDB_MEMORY_LIMIT=8GB
export PGX_TARGET_FILE_SIZE_MB=1024
export PGX_MAX_CHUNKS_PER_BATCH=8
export PGX_NO_MERGE=0
export PGX_PERSIST_MAPPINGS=1
export PGX_SKIP_SAMPLE_CHECK=1
export PGX_USE_TEMP_DB=1
export PGX_MAX_UPLOAD_CONCURRENCY=10
export PGX_S3_MAX_CONNECTIONS=256
export PGX_MP_START_METHOD=fork


# ========================================
# CHUNKING: Balanced for performance
# ========================================
CHUNK_ROWS=1000000  # 1M rows per chunk (good balance)
CHECKPOINT_SUFFIX=".codes_updated.v2.ok"
STAGING_SUFFIX=".codes_updated.staging/"

# ========================================
# PATHS
# ========================================
ICD_MAP="/home/pgx3874/pgx-analysis/1_apcd_input_data/target_mapping/target_icd_mapping.json"
LOG_FILE="logs/medical_codes_$(date +%Y%m%d_%H%M%S).log"

# ========================================
# SETUP
# ========================================
mkdir -p logs

echo "🚀 Starting with LOCAL STAGING for maximum performance"
echo "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"
echo "📂 Staging dir: $PGX_LOCAL_STAGING_DIR"
echo "👷 Workers: $PGX_WORKERS_MEDICAL"
echo "🧠 Memory per worker: $PGX_DUCKDB_MEMORY_LIMIT"
echo "📦 Chunk size: $CHUNK_ROWS rows"
echo "📋 Log: $LOG_FILE"
echo "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"
echo "Started at: $(date)"
echo ""

# ========================================
# RUN
# ========================================
nohup /home/pgx3874/jupyter-env/bin/python3.11 \
  /home/pgx3874/pgx-analysis/1_apcd_input_data/7_update_codes.py \
  --icd-target-map "$ICD_MAP" \
  --years "2016,2017,2018,2019,2020" \
  --workers-medical $PGX_WORKERS_MEDICAL \
  --threads $PGX_THREADS_PER_WORKER \
  --resume \
  --checkpoint-suffix "$CHECKPOINT_SUFFIX" \
  --chunked \
  --chunk-rows $CHUNK_ROWS \
  --staging-suffix "$STAGING_SUFFIX" \
  --duckdb-mem-limit $PGX_DUCKDB_MEMORY_LIMIT \
  --no-merge \
  > "$LOG_FILE" 2>&1 &

echo $! > logs/medical_codes.pid
echo "✅ Job started with PID: $(cat logs/medical_codes.pid)"
echo ""
echo "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"
echo "MONITOR PROGRESS:"
echo "  !tail -f $LOG_FILE"
echo "  !grep -c '✓ Updated' $LOG_FILE"
echo "  !du -sh /mnt/nvme/duckdb_tmp"
echo "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"


In [ ]:
import glob
import os
import subprocess

log_files = glob.glob("logs/medical_codes_*.log")
if log_files:
    latest_log = max(log_files, key=os.path.getmtime)
    
    result = subprocess.run(
        ['grep', '-c', '✓ Updated', latest_log],
        capture_output=True,
        text=True
    )
    completed = result.stdout.strip() if result.returncode == 0 else "0"
    
    # Disk usage
    staging_size = subprocess.check_output(
        ['du', '-sh', '/mnt/nvme/duckdb_tmp'],
        stderr=subprocess.DEVNULL
    ).decode().strip().split()[0]
    
    duckdb_tmp_size = subprocess.check_output(
        ['du', '-sh', '/mnt/nvme/duckdb_tmp'],
        stderr=subprocess.DEVNULL
    ).decode().strip().split()[0]
    
    print(f"✓ Completed: {completed} partitions")
    print(f"💾 Staging: {staging_size}")
    print(f"💾 DuckDB tmp: {duckdb_tmp_size}")
    print(f"📋 Log: {os.path.basename(latest_log)}")

### QA/Validate Updates

#### 1. Re-Run Frequency Analysis

In [ ]:
%%bash
export PGX_WORKERS_MEDICAL=16
export PGX_THREADS_PER_WORKER=1
export PGX_S3_MAX_CONNECTIONS=64  # optional
                             
/home/pgx3874/jupyter-env/bin/python3.11 \
  /home/pgx3874/pgx-analysis/1_apcd_input_data/6_target_frequency_analysis.py \
  --codes-of-interest "F11.20" \
  --workers ${PGX_WORKERS_MEDICAL} \
  --min-year 2016 --max-year 2020 \
  --log-cpu --log-s3

#### 2. Updated Pickle Files

In [ ]:
import os
import pandas as pd
from helpers_1997_13.data_utils import (
    load_target_artifacts,
    find_variants,
    totals_for_codes,
    compare_totals,
)

# Where scripts write canonical artifacts
outputs_dir = os.path.join('1_apcd_input_data', 'outputs')
legacy_base = '1_apcd_input_data'

# Load normalized artifacts. This will raise FileNotFoundError if the
# canonical updated pickle is missing — generate it with:
#   python 1_apcd_input_data/6_target_frequency_analysis.py --codes-of-interest "F11.20"
t_orig, t_updated = load_target_artifacts(outputs_dir=outputs_dir)

print('Updated data shape:', getattr(t_updated, 'shape', None))
print('Orig data shape:', getattr(t_orig, 'shape', None))

# Summary comparison (orig vs updated totals)
cmp = compare_totals(t_orig, t_updated)
print('\nTop 10 updated target_codes (by updated_total):')
print(cmp.head(10).to_string(index=False))

# Focused QA for a code of interest
code_of_interest = 'F11.20'
orig_variants = find_variants(t_orig, code_of_interest)
upd_variants = find_variants(t_updated, code_of_interest)
all_variants = sorted(set(orig_variants) | set(upd_variants))

print(f"\nF11.20 variants detected - orig: {len(orig_variants)}, updated: {len(upd_variants)}, union: {len(all_variants)}")
print('Variants union:', all_variants)

# Per-variant totals and CSV output
o = totals_for_codes(t_orig, all_variants).rename(columns={'freq': 'orig_freq'})
u = totals_for_codes(t_updated, all_variants).rename(columns={'freq': 'updated_freq'})
summary = pd.merge(pd.DataFrame({'target_code': all_variants}), o, on='target_code', how='left').merge(u, on='target_code', how='left').fillna(0)
summary['delta'] = summary['updated_freq'].astype(int) - summary['orig_freq'].astype(int)

out_csv = os.path.join(outputs_dir, 'target_code_f1120_comparison.csv')
os.makedirs(outputs_dir, exist_ok=True)
summary.to_csv(out_csv, index=False)
print('\nSaved comparison CSV to', out_csv)

In [ ]:
import pickle
import pandas as pd
from helpers_1997_13.data_utils import normalize_to_all_targets

p='1_apcd_input_data/outputs'
with open(f'{p}/target_analysis_data.pkl','rb') as f:
    orig_obj = pickle.load(f)
with open(f'{p}/target_analysis_data_updated.pkl','rb') as f:
    updated_obj = pickle.load(f)

o_df = normalize_to_all_targets(orig_obj)
u_df = normalize_to_all_targets(updated_obj)

# Show counts for the two variants
print("Orig counts:")
print(o_df[o_df['target_code'].str.contains('AF1120|F1120', na=False)]
      .groupby('target_code')['frequency'].sum())

print("\nUpdated counts:")
print(u_df[u_df['target_code'].str.contains('AF1120|F1120', na=False)]
      .groupby('target_code')['frequency'].sum())

#### 3. F1120 Frequency Chart

In [ ]:
# Target code (ICD) == F11.20 (include variants like YF1120, 0F1120, etc.)

# u_df should have: event_year, target_code, frequency, target_system
# If loaded from pickle produced by 6_target_frequency_analysis.py:
#   u_df = target_data['all_targets']

code_of_interest = 'F11.20'
needle = code_of_interest.replace('.', '').upper()

icd = u_df[u_df['target_system'] == 'icd'].copy()
icd['code_flat'] = (
    icd['target_code'].astype(str)
       .str.upper()
       .str.replace('.', '', regex=False)
       .str.replace(' ', '', regex=False)
)

# All variants containing the same flattened substring (F1120)
codes = (
    icd[icd['code_flat'].str.contains(needle, na=False)]
    .groupby('target_code', as_index=False)['frequency'].sum()
    .sort_values('frequency', ascending=False)['target_code']
    .tolist()
)

from helpers_1997_13.visualization_utils import plot_icd_variant_heatmap, save_current_chart

fig, pivot_df = plot_icd_variant_heatmap(
    u_df=target_data['all_targets'],
    code_of_interest='F1120',   # already flattened/normalized
    save_path='plots/icd_F11.20_heatmap.png',
    export_csv_path='plots/icd_F11.20_pivot.csv'
)


In [ ]:
import os
import pickle
import json
import pandas as pd
import duckdb

# Paths (adjust if your environment differs)
base = '/home/pgx3874/pgx-analysis/1_apcd_input_data'
orig_pk = os.path.join(base, 'target_code_analysis_data.orig.pkl')
updated_pk = os.path.join(base, 'target_code_analysis_data.updated.pkl')
out_json = '/home/pgx3874/pgx-analysis/docs/target_pickles_diff.json'

def load_pickle(path):
  if not os.path.exists(path):
    return None
  try:
    with open(path, 'rb') as f:
      return pickle.load(f)
  except Exception as e:
    print(f"⚠️ Failed to load pickle {path}: {e}")
    return None

def normalize_to_all_targets(obj):
  # Same normalizer used elsewhere: produce columns event_year, target_code, frequency, target_system
  if obj is None:
    return None
  if isinstance(obj, pd.DataFrame):
    df = obj.copy()
    for c in ['event_year','target_code','frequency','target_system']:
      if c not in df.columns:
        df[c] = pd.NA
    return df[['event_year','target_code','frequency','target_system']]
  if isinstance(obj, dict):
    if 'all_targets' in obj and obj['all_targets'] is not None:
      return normalize_to_all_targets(obj['all_targets'])
    parts = []
    if obj.get('icd_aggregated') is not None:
      parts.append(obj['icd_aggregated'].assign(target_system='icd'))
    if obj.get('cpt_aggregated') is not None:
      parts.append(obj['cpt_aggregated'].assign(target_system='cpt'))
    if parts:
      out = pd.concat(parts, ignore_index=True)
      for c in ['event_year','target_code','frequency','target_system']:
        if c not in out.columns:
          out[c] = pd.NA
      return out[['event_year','target_code','frequency','target_system']]
    dfs = [v for v in obj.values() if isinstance(v, pd.DataFrame)]
    if dfs:
      out = pd.concat(dfs, ignore_index=True)
      for c in ['event_year','target_code','frequency','target_system']:
        if c not in out.columns:
          out[c] = pd.NA
      return out[['event_year','target_code','frequency','target_system']]
  return None

pd_orig = load_pickle(orig_pk)
pd_updated = load_pickle(updated_pk)

t_orig = normalize_to_all_targets(pd_orig) or pd.DataFrame(columns=['event_year','target_code','frequency','target_system'])
t_updated = normalize_to_all_targets(pd_updated) or pd.DataFrame(columns=['event_year','target_code','frequency','target_system'])

# Coerce numeric types
for df in (t_orig, t_updated):
  if 'frequency' in df.columns:
    df['frequency'] = pd.to_numeric(df['frequency'], errors='coerce').fillna(0).astype(int)
  if 'event_year' in df.columns:
    df['event_year'] = pd.to_numeric(df['event_year'], errors='coerce').fillna(0).astype(int)

def totals_by_code(df):
  if df is None or df.empty:
    return {}
  grp = df.groupby('target_code', as_index=False)['frequency'].sum()
  return {str(r['target_code']): int(r['frequency']) for _, r in grp.iterrows()}

def per_year_by_code(df):
  if df is None or df.empty:
    return {}
  out = {}
  for code, g in df.groupby('target_code'):
    series = g.groupby('event_year')['frequency'].sum()
    out[str(code)] = {str(int(k)): int(v) for k, v in series.to_dict().items()}
  return out

orig_totals = totals_by_code(t_orig)
upd_totals = totals_by_code(t_updated)
orig_by_year = per_year_by_code(t_orig)
upd_by_year = per_year_by_code(t_updated)

all_codes = sorted(set(list(orig_totals.keys()) + list(upd_totals.keys())))

records = []
for code in all_codes:
  o = orig_totals.get(code, 0)
  u = upd_totals.get(code, 0)
  rec = {
    'target_code': code,
    'orig_total': int(o),
    'updated_total': int(u),
    'delta': int(u) - int(o),
    'orig_by_year': orig_by_year.get(code, {}),
    'updated_by_year': upd_by_year.get(code, {}),
  }
  records.append(rec)

# Focused F11.20 variants (same heuristic as Cell 35)
code_of_interest = 'F11.20'
needle = code_of_interest.replace('.', '').upper()
def find_variants(df):
  if df is None or df.empty:
    return []
  tmp = df.copy()
  tmp['code_flat'] = tmp['target_code'].astype(str).str.upper().str.replace('.', '', regex=False).str.replace(' ', '', regex=False)
  codes = tmp[tmp['code_flat'].str.contains(needle, na=False)].groupby('target_code', as_index=False)['frequency'].sum().sort_values('frequency', ascending=False)['target_code'].tolist()
  return [str(c) for c in codes]

f_orig = find_variants(t_orig)
f_upd = find_variants(t_updated)
f_union = sorted(set(f_orig) | set(f_upd))

diff_obj = {
  'metadata': {
    'orig_shape': list(t_orig.shape),
    'updated_shape': list(t_updated.shape),
  },
  'totals': records,
  'f11_20_variants': {
    'orig': f_orig,
    'updated': f_upd,
    'union': f_union
  }
}

# Write JSON
os.makedirs(os.path.dirname(out_json), exist_ok=True)
with open(out_json, 'w', encoding='utf-8') as f:
  json.dump(diff_obj, f, indent=2, ensure_ascii=False)

print('Wrote JSON diff to', out_json)

#### 4. Final Interactive Target Code Dashboard

In [ ]:
from helpers_1997_13.visualization_utils import create_plotly_frequency_dashboard

# Target Code Dashboard (show all; user filters interactively)
create_plotly_frequency_dashboard(
    u_df,
    title='Target Frequency Explorer (ICD/CPT)',
    s3_output_path='s3://pgxdatalake/visualizations/target_code/target_frequency_dashboard.html',
    target_col='target_code',
    year_col='event_year',
    freq_col='frequency',
    system_col='target_system',
    top_n=999999  # effectively all
)

## B. Build Cohorts (From Medical and Pharmacy)

In [ ]:
%%bash
# Checkpoints
aws s3 rm s3://pgx-repository/pgx-pipeline-status/create_cohort/ --recursive

# Outputs
aws s3 rm s3://pgxdatalake/gold/cohorts_F1120/ --recursive

### 1. Merge Datasets/Build Cohorts

#### a. All Cohorts

In [ ]:
import sys
import subprocess
import concurrent.futures
import boto3
import traceback
import os
import threading
import queue
import time

from helpers_1997_13 import constants
from helpers_1997_13.cohort_utils import check_existing_cohorts as cu_check_existing_cohorts, run_cohort as cu_run_cohort
import functools

# Script configuration
# Adjust this path to where 0_create_cohort.py lives on the target host
script_path = "/home/pgx3874/pgx-analysis/2_create_cohort/0_create_cohort.py"
python_bin = sys.executable


# Use shared helpers from helpers_1997_13.cohort_utils to avoid duplication.
def check_existing_cohorts():
    return cu_check_existing_cohorts()


# Wrapper that binds script_path and python_bin into the shared runner.
def run_cohort(job):
    target_icd = os.environ.get('PGX_TARGET_ICD_CODES', 'F1120')
    cb = functools.partial(cu_run_cohort, script_path=script_path, python_bin=python_bin, target_icd=target_icd)
    return cb(job)


# ----- Batch processing orchestration -----

if __name__ == '__main__':
    jobs_to_process = check_existing_cohorts()

    if not jobs_to_process:
        print("\nAll cohorts already exist or are locked. No jobs to run.")
        sys.exit(0)

    MAX_WORKERS = min(2, len(jobs_to_process))
    print(f"\nStarting {len(jobs_to_process)} cohort processing jobs with {MAX_WORKERS} parallel workers...", flush=True)
    print(f"{'='*80}", flush=True)

    BATCH_SIZE = 2
    all_job_batches = [jobs_to_process[i:i+BATCH_SIZE] for i in range(0, len(jobs_to_process), BATCH_SIZE)]
    all_results = []
    all_job_statuses = {}

    for batch_num, job_batch in enumerate(all_job_batches, 1):
        print(f"\nProcessing batch {batch_num}/{len(all_job_batches)} with {len(job_batch)} jobs...", flush=True)
        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            future_to_job = {executor.submit(run_cohort, job): job for job in job_batch}
            total_jobs = len(future_to_job)
            completed = 0
            for future in concurrent.futures.as_completed(future_to_job):
                job = future_to_job[future]
                job_id = f"{job['age_band']}/{job['event_year']}"
                completed += 1
                try:
                    result = future.result()
                    all_results.append(result)
                    all_job_statuses[job_id] = result
                    print(f"\n[{completed}/{total_jobs}] Job status: {result}", flush=True)
                except Exception as e:
                    error_msg = f"Job execution error for {job_id}: {str(e)}"
                    print(f"\n[{completed}/{total_jobs}] {error_msg}", flush=True)
                    print(traceback.format_exc(), flush=True)
                    all_results.append(f"ERROR: {job_id} - {str(e)}")
                    all_job_statuses[job_id] = f"ERROR: {str(e)}"

                print(f"\nProgress: {completed}/{total_jobs} jobs completed ({100*completed/total_jobs:.1f}%)", flush=True)
                print(f"{'='*80}", flush=True)

        print(f"\nBatch {batch_num}/{len(all_job_batches)} complete.", flush=True)
        if batch_num < len(all_job_batches):
            print("Pausing briefly before starting next batch...", flush=True)
            time.sleep(5)

    print("\n" + "="*80)
    print("FINAL SUMMARY OF RESULTS:")
    print("="*80)
    success_count = sum(1 for r in all_results if r.startswith("SUCCESS"))
    locked_count = sum(1 for r in all_results if r.startswith("SKIPPED_LOCKED"))
    failed_count = sum(1 for r in all_results if r.startswith("FAILED") or r.startswith("ERROR"))

    print(f"✓ Successful: {success_count}")
    print(f"⚠ Skipped (locked): {locked_count}")
    print(f"✗ Failed: {failed_count}")

    if all_job_statuses:
        print("\nDetailed status by job:")
        for job_id, status in sorted(all_job_statuses.items()):
            if status.startswith("SUCCESS"):
                status_icon = "✓"
            elif status.startswith("SKIPPED"):
                status_icon = "⚠"
            else:
                status_icon = "✗"
            print(f"{status_icon} {job_id}: {status}")

    print("="*80)


### 2. Glue Crawler for Athena

In [ ]:
import boto3

# Initialize a Glue client (default region or specify if needed)
glue = boto3.client('glue', region_name='us-east-1')  # change region if necessary

# Name of your crawler
crawler_name = "medical"

# Start the crawler
response = glue.start_crawler(Name=crawler_name)

# Print response for confirmation
print(f"Crawler '{crawler_name}' started successfully.")
print(response)


### 3. QA

In [ ]:
%%bash
set -euo pipefail

# Set target configuration environment variables (matching 0_create_cohort.py)  
export PGX_TARGET_ICD_CODES="${PGX_TARGET_ICD_CODES:-F1120}"
export PGX_TARGET_NAME="${PGX_TARGET_NAME:-}"
export PGX_TARGET_CPT_CODES="${PGX_TARGET_CPT_CODES:-}"
export PGX_TARGET_ICD_PREFIXES="${PGX_TARGET_ICD_PREFIXES:-}"
export PGX_TARGET_CPT_PREFIXES="${PGX_TARGET_CPT_PREFIXES:-}"

/home/pgx3874/jupyter-env/bin/python3.11 \
  /home/pgx3874/pgx-analysis/2_create_cohort/2_step2_data_quality_qa.py \
  --all-age-bands \
  --all-event-years \
  --cohorts both \
  --save-results \
  --max-workers 16 \
  --log-level INFO

### 4. Athena Queries

In [ ]:
import boto3
import awswrangler as wr

session = boto3.Session(region_name="us-east-1")

# Set it for awswrangler
wr.config.athena_workgroup = "primary"
wr.config.athena_output_location = "s3://aws-athena-query-results-us-east-1-535362115856/cohort-qa/"
wr.config.session = session


In [ ]:
%load_ext sql
%config SqlMagic.autocommit=False

In [ ]:
query = """
SELECT *
FROM cohorts.cohorts_F1120
WHERE cohort_name = 'non_opioid_ed'
  AND age_band = '65-74'
  AND event_year = '2016'
LIMIT 5
"""
wr.athena.read_sql_query(sql=query, database="cohorts", boto3_session=session)


In [ ]:
query = """
SELECT * 
FROM cohorts.cohorts_clean
WHERE cohort_name = 'non_opioid_ed' 
  AND age_band = '65-74' 
  AND event_year = '2016'
LIMIT 5
"""
wr.athena.read_sql_query(sql=query, database="cohorts", boto3_session=session)


In [ ]:
query_1 = """
SELECT 
  cohort_name,
  COUNT(*) AS total_rows,
  COUNT(DISTINCT mi_person_key) AS distinct_patients
FROM cohorts.cohorts_clean
GROUP BY cohort_name
"""

df_1 = wr.athena.read_sql_query(sql=query_1, database="cohorts", boto3_session=session)
df_1

In [ ]:
query_2 = """
SELECT first_event, event, COUNT(*) AS count
FROM cohorts.cohorts_clean
WHERE cohort_name = 'non_opioid_ed' 
  AND age_band = '65-74' 
  AND event_year = '2016'
GROUP BY first_event, event
ORDER BY count DESC
"""

df_2 = wr.athena.read_sql_query(sql=query_2, database="cohorts", boto3_session=session)
df_2

In [ ]:
query_3 = """
SELECT COUNT(*) AS patients_with_many_rows
FROM (
  SELECT mi_person_key
  FROM cohorts.cohorts_clean
  WHERE cohort_name = 'non_opioid_ed' 
    AND age_band = '65-74' 
    AND event_year = '2016'
  GROUP BY mi_person_key
  HAVING COUNT(*) > 100
)
"""

df_3 = wr.athena.read_sql_query(query_3, database="cohorts", boto3_session=session)
df_3

In [ ]:
query_1 = """
SELECT
  cohort_name,
  COUNT(*) AS total_rows,
  COUNT(DISTINCT mi_person_key) AS distinct_patients
FROM cohorts.cohorts_clean
GROUP BY cohort_name
"""

df_1 = wr.athena.read_sql_query(sql=query_1, database="cohorts", boto3_session=session)
df_1


In [ ]:
query_4 = """
SELECT COUNT(*) AS patients_with_many_rows
FROM (
  SELECT mi_person_key
  FROM cohorts.cohorts_clean
  WHERE cohort_name = 'opioid_ed' 
    AND age_band = '65-74' 
    AND event_year = '2016'
  GROUP BY mi_person_key
  HAVING COUNT(*) > 100
)
"""

df_4 = wr.athena.read_sql_query(query_4, database="cohorts", boto3_session=session)
df_4

In [ ]:
query_2 = """
SELECT first_event, event, COUNT(*) AS count
FROM cohorts.cohorts_clean
WHERE cohort_name = 'non_opioid_ed' 
  AND age_band = '65-74' 
  AND event_year = '2016'
GROUP BY first_event, event
ORDER BY count DESC
"""

df_2 = wr.athena.read_sql_query(sql=query_2, database="cohorts", boto3_session=session)
df_2


In [ ]:
query_4 = """
SELECT *
FROM cohorts.cohorts_clean
WHERE cohort_name = 'non_opioid_ed' 
  AND age_band = '65-74' 
  AND event_year IN ('2016', '2017', '2018')
"""

train_df = wr.athena.read_sql_query(query_4, database="cohorts", boto3_session=session)
train_df.head()


In [ ]:
query_5 = """
SELECT *
FROM cohorts.cohorts_clean
WHERE cohort_name = 'non_opioid_ed' 
  AND age_band = '65-74' 
  AND event_year = '2019'
"""

test_df = wr.athena.read_sql_query(query_5, database="cohorts", boto3_session=session)
test_df.head()


In [ ]:
query_5 = """
SELECT *
FROM cohorts.cohorts_clean
WHERE cohort_name = 'opioid_ed' 
  AND age_band = '65-74' 
  AND event_year = '2019'
"""

test_df = wr.athena.read_sql_query(query_5, database="cohorts", boto3_session=session)
test_df.head()


In [ ]:
query_4 = """
SELECT *
FROM cohorts.cohorts_clean
WHERE cohort_name = 'non_opioid_ed' 
  AND age_band = '65-74' 
  AND event_year IN ('2016', '2017', '2018')
"""

train_df = wr.athena.read_sql_query(query_4, database="cohorts", boto3_session=session)
train_df.head()


In [ ]:
query = """
SELECT first_event, COUNT(*) AS patient_count
FROM cohorts.cohorts_clean
GROUP BY first_event
ORDER BY patient_count DESC
"""

df = wr.athena.read_sql_query(
    sql=query,
    database="cohorts",
    boto3_session=session
)

df.head()

In [ ]:
query_5 = """
SELECT *
FROM cohorts.cohorts_clean
WHERE cohort_name = 'non_opioid_ed'
  AND age_band = '65-74'
  AND event_year = '2019'
"""

test_df = wr.athena.read_sql_query(query_5, database="cohorts", boto3_session=session)
test_df.head()


In [ ]:
query = """
SELECT event, COUNT(*) AS patient_count
FROM cohorts.cohorts_clean
GROUP BY event
ORDER BY patient_count DESC
"""

df = wr.athena.read_sql_query(
    sql=query,
    database="cohorts",
    boto3_session=session
)

df.head()

In [ ]:
query_6 = """
SELECT *
FROM cohorts.cohorts_clean
WHERE cohort_name = 'opioid_ed' 
  AND age_band = '65-74' 
  AND event_year = '2019'
"""

test_df = wr.athena.read_sql_query(query_6, database="cohorts", boto3_session=session)
test_df.head()


In [ ]:
query = """
SELECT age_band, event_year, COUNT(*) AS null_or_empty_drug_rows
FROM pharmacy.pharmacy_clean
WHERE drug_name = '' OR drug_name IS NULL
GROUP BY age_band, event_year
ORDER BY null_or_empty_drug_rows DESC
"""

control_df_missing = wr.athena.read_sql_query(query, database="pharmacy", boto3_session=session)
control_df_missing


## C. Feature Engineer (Network Analysis)

### S3 directory structure and pipeline flow

- Global FP-Growth (ProcessPool 30)
  - Input: `s3://pgxdatalake/silver/{pharmacy|medical}/age_band={band}/event_year={year}/*.parquet`
  - Output (GOLD): `s3://pgxdatalake/gold/fpgrowth/global/`
    - `itemsets_{age}_{period_or_year}.json`
    - `rules_{age}_{period_or_year}.json`
    - `drug_metrics_{age}_{period_or_year}.json`
    - `drug_encoding_map_{age}_{period_or_year}.json`

- Cohort FP-Growth (ProcessPool 30)
  - Input (curated cohort start): `s3://pgxdatalake/gold/cohorts_clean/cohort_name={cohort}/age_band={band}/event_year={year}/cohort.parquet`
  - Output (GOLD): `s3://pgxdatalake/gold/fpgrowth/cohort/cohort_name={cohort}/age_band={band}/event_year={year}/`
    - `fpgrowth_features.parquet`
    - `itemsets.parquet` and `itemsets.json`
    - `rules.parquet` and `rules.json`
    - `feature_manifest.json`
    - `*_drug_network.html`

- Sources summary
  - Reads: SILVER medical/pharmacy for global; GOLD cohorts_clean for cohort
  - Writes: GOLD fpgrowth/{global|cohort} and GOLD cohorts_clean for curated cohort parquet

```
S3 Results:
├── gold/fpgrowth/global/
│   ├── itemsets_*.json
│   ├── rules_*.json
│   ├── drug_metrics_*.json
│   └── drug_encoding_map_*.json
├── gold/fpgrowth/cohort/
│   └── cohort_name={cohort}/age_band={age}/event_year={year}/
│       ├── fpgrowth_features.parquet
│       ├── itemsets.parquet
│       ├── itemsets.json
│       ├── rules.parquet
│       ├── rules.json
│       ├── {cohort}_{age}_{year}_drug_network.html
│       └── feature_manifest.json
└── gold/cohorts_clean/
    └── cohort_name={cohort}/age_band={age}/event_year={year}/cohort.parquet
```

📋 S3 Path Mapping Summary

| Component | Path |
|---|---|
| Global inputs | `s3://pgxdatalake/silver/{pharmacy|medical}/age_band={band}/event_year={year}/*.parquet` |
| Global outputs (GOLD) | `s3://pgxdatalake/gold/fpgrowth/global/{itemsets|rules|drug_metrics|drug_encoding_map}_*.json` |
| Cohort input (curated) | `s3://pgxdatalake/gold/cohorts_clean/cohort_name={cohort}/age_band={band}/event_year={year}/cohort.parquet` |
| Cohort outputs (GOLD) | `s3://pgxdatalake/gold/fpgrowth/cohort/cohort_name={cohort}/age_band={band}/event_year={year}/...` |


### FP-Growth Analysis Module Summary

#### 📋 Overview
The FP-Growth Analysis Module implements comprehensive drug pattern mining using the Frequent Pattern Growth algorithm. This module provides **two complementary approaches** for drug association analysis:

#### 🎯 Dual Approach Strategy

##### 1. **Global FP-Growth** → CatBoost Feature Engineering
- **Purpose**: Creates universal drug encoding features that work across all cohorts
- **Why Essential**: CatBoost models require consistent feature spaces - same drug encodings across training/validation/test sets
- **Output**: Global drug encoding map (`s3://pgxdatalake/gold/fpgrowth/global/drug_encoding_map.json`)
- **Benefit**: Population-level drug pattern insights become numerical features for ML

##### 2. **By-Cohort FP-Growth** → BupaR Process Mining  
- **Purpose**: Discovers cohort-specific drug patterns and treatment sequences
- **Why Essential**: Different cohorts (ED vs non-ED, age groups) have fundamentally different care pathways
- **Output**: Cohort-specific association rules and network visualizations
- **Benefit**: Reveals how prescribing patterns differ between populations

#### 🔧 Technical Implementation

##### Core Components:
- **Global Pipeline**: `run_fpgrowth_global()` - Creates universal encoding features
- **Cohort Pipeline**: `process_features()` - Discovers cohort-specific patterns
- **Validation**: Cross-pipeline validation and integration testing
- **Storage**: S3-based results partitioned by cohort/age/year

##### Key Parameters:
- **Global Support**: 0.005 (lower threshold for population coverage)
- **Cohort Support**: 0.05 (higher threshold for pattern significance)
- **Min Confidence**: 0.01 for association rules
- **Top-K**: 25 most frequent itemsets per cohort

##### 📊 Output Structure

```
S3 Results:
├── global_fpgrowth/
│   ├── drug_encoding_map.json     # Universal ML features
│   ├── global_itemsets.json       # Population patterns
│   └── global_rules.json          # Association rules
└── fpgrowth_features/             # Cohort-specific results
    ├── cohort_name=ed_non_opioid/
    │   ├── age_band=25-34/
    │   └── age_band=35-44/
    └── cohort_name=opioid_ed/
```

#### 🔗 Integration Points

##### With CatBoost Models:
- Global encoding map provides consistent categorical features
- Drug patterns become numerical features for machine learning
- Population-level insights enhance model generalization

##### With BupaR Process Mining:
- Cohort-specific patterns reveal treatment pathways
- Association rules guide process flow analysis
- Network visualizations show care transitions

#### 📈 Pipeline Workflow
1. **Extract** → Get unique drug names from pharmacy dataset
2. **Transform** → Create patient-level drug transactions  
3. **Mine** → Apply FP-Growth with appropriate support thresholds
4. **Encode** → Generate consistent feature encodings
5. **Store** → Save results partitioned for downstream consumption
6. **Validate** → Ensure integration readiness for ML and process mining

---

#### 📋 S3 Path Mapping Summary

The following table shows how FP-Growth outputs map to downstream analysis inputs:

| **Component** | **FP-Growth Output Path** | **Downstream Input** | **Purpose** |
|---------------|---------------------------|----------------------|-------------|
| **Global Encoding** | `s3://pgxdatalake/global_fpgrowth/drug_encoding_map.json` | CatBoost Feature Engineering | Universal drug encodings for consistent ML features |
| **Global Itemsets** | `s3://pgxdatalake/global_fpgrowth/global_itemsets.json` | CatBoost Model Training | Population-level drug patterns |
| **Global Rules** | `s3://pgxdatalake/global_fpgrowth/global_rules.json` | CatBoost Model Training | Association rules for feature creation |
| **Enhanced Features** | `s3://pgxdatalake/fpgrowth_features_parquet/cohort_name={}/age_band={}/event_year={}/fpgrowth_features.parquet` | CatBoost Training Data | Dataset with `pattern_*` columns and FP-Growth metrics |
| **Cohort Itemsets** | `s3://pgxdatalake/itemsets_parquet/cohort_name={}/age_band={}/event_year={}/itemsets.parquet` | BupaR Process Mining | Cohort-specific frequent drug combinations |
| **Cohort Rules** | `s3://pgxdatalake/rules_parquet/cohort_name={}/age_band={}/event_year={}/rules.parquet` | BupaR Process Mining | Cohort-specific association rules for pathway analysis |
| **Network Visualizations** | `s3://pgxdatalake/html_network_visual/cohort_name={}/age_band={}/event_year={}/network_visualization.html` | BupaR Analysis | Drug interaction network graphs |

#### 🔗 Integration Flow

```mermaid
graph LR
    A[FP-Growth Global Pipeline] --> B[global_fpgrowth/*]
    C[FP-Growth Cohort Pipeline] --> D[fpgrowth_features_parquet/*]
    C --> E[itemsets_parquet/*]
    C --> F[rules_parquet/*]
    
    B --> G[CatBoost Global Features]
    D --> H[CatBoost Enhanced Dataset]
    E --> I[BupaR Itemsets Analysis]
    F --> J[BupaR Rules Analysis]
    
    G --> K[Universal Drug Encodings]
    H --> L[Pattern_* Columns]
    I --> M[Drug Combination Discovery]
    J --> N[Treatment Pathway Analysis]
```

#### ⚠️ Critical Dependencies

1. **CatBoost Requirements**:
   - Global encoding map must be available before model training
   - Enhanced dataset must contain `pattern_*`, `*_enc_*`, `*_support`, `*_confidence`, `*_lift` columns
   - Pattern columns should be categorical (string) features
   - Metric columns should be numerical (float) features

2. **BupaR Requirements**:
   - Itemsets data needed for frequent drug combination analysis
   - Rules data needed for treatment pathway discovery
   - Both must be available per cohort/age_band/event_year partition

3. **Data Consistency**:
   - Same `cohort_name`, `age_band`, `event_year` combinations across all output types
   - Pattern encoding consistency between global and cohort-specific results

### FPGrowth Global Metrics

In [ ]:
!/home/pgx3874/jupyter-env/bin/python3.11 /home/pgx3874/pgx-analysis/fpgrowth_analysis/run_fpgrowth_group_pipeline.py 2>&1 | tee global_group_fpgrowth_output.log

#### Global Encoded Features

In [ ]:
!/home/pgx3874/jupyter-env/bin/python3.11 /home/pgx3874/pgx-analysis/fpgrowth_analysis/process_itemsets.py 2>&1 | tee encode_drug_name_output.log

#### 📊 Drug Encoding Data Structure

| Column | Description |
|---|---|
| `drug_name` | Original drug name |
| `global_encoded_name` | 29-character encoded drug name |
| `first_letter_index` | First letter index (A=111, B=222, etc.) |
| `word_length` | Word length |
| `num_vowels` | Number of vowels |
| `num_consonants` | Number of consonants |
| `num_hyphens_underscores` | Compound drug indicators |
| `num_chemical_suffixes` | Chemical suffix count (e.g., -ine, -ol, -ate, -ide) |
| `num_consonant_clusters` | Count of consecutive consonant clusters |
| `repetition_factor` | Count of repeated letters |
| `support` | FP-Growth support metric |
| `num_rules` | Number of association rules containing the drug |
| `num_drugs_in_rules` | Total drugs across rules involving the drug |
| `trend` | Trend slope over time (support) |
| `trend_direction` | Increasing/decreasing/stable trend |

In [ ]:
import boto3
import pandas as pd
from io import BytesIO

# One-liner to get latest CSV and view first 10 rows with key columns
s3_client = boto3.client('s3')
response = s3_client.list_objects_v2(Bucket='pgxdatalake', Prefix='pgx_pipeline/fpgrowth_analysis/processed_itemsets/drug_names/')
csv_files = [obj for obj in response.get('Contents', []) if 'drug_names_with_feature_metrics' in obj['Key'] and obj['Key'].endswith('.csv')]
latest_file = max(csv_files, key=lambda x: x['LastModified'])['Key']
response = s3_client.get_object(Bucket='pgxdatalake', Key=latest_file)
df = pd.read_csv(BytesIO(response['Body'].read()), nrows=10)

# Select key columns: drug_name, global_encoded_name, linguistic features, and metrics
key_columns = [
    'drug_name',                    # Original drug name
    'global_encoded_name',          # 29-character encoded drug name
    'first_letter_index',           # First letter index (A=111, B=222, etc.)
    'word_length',                  # Word length
    'num_vowels',                   # Number of vowels
    'num_consonants'              # Number of consonants
]
available_columns = [col for col in key_columns if col in df.columns]
selected_df = df[available_columns]

print(f"Latest file: {latest_file}")
print(f"Showing columns: {available_columns}")
print("=" * 100)
print(selected_df.to_string(index=False))

In [ ]:
import boto3
import pandas as pd
from io import BytesIO

# One-liner to get latest CSV and view first 10 rows with key columns
s3_client = boto3.client('s3')
response = s3_client.list_objects_v2(Bucket='pgxdatalake', Prefix='pgx_pipeline/fpgrowth_analysis/processed_itemsets/drug_names/')
csv_files = [obj for obj in response.get('Contents', []) if 'drug_names_with_feature_metrics' in obj['Key'] and obj['Key'].endswith('.csv')]
latest_file = max(csv_files, key=lambda x: x['LastModified'])['Key']
response = s3_client.get_object(Bucket='pgxdatalake', Key=latest_file)
df = pd.read_csv(BytesIO(response['Body'].read()), nrows=10)

# Select key columns: drug_name, global_encoded_name, linguistic features, and metrics
key_columns = [
    'drug_name',                    # Original drug name
    'global_encoded_name',          # 29-character encoded drug name
    'num_hyphens_underscores',      # Compound drug indicators
    'num_chemical_suffixes',        # Chemical suffix count (-ine, -ol, -ate, etc.)
    'num_consonant_clusters',       # Consecutive consonants
    'repetition_factor'          # Repeated letters
]
available_columns = [col for col in key_columns if col in df.columns]
selected_df = df[available_columns]

print(f"Latest file: {latest_file}")
print(f"Showing columns: {available_columns}")
print("=" * 100)
print(selected_df.to_string(index=False))

In [ ]:
import boto3
import pandas as pd
from io import BytesIO

# One-liner to get latest CSV and view first 10 rows with key columns
s3_client = boto3.client('s3')
response = s3_client.list_objects_v2(Bucket='pgxdatalake', Prefix='pgx_pipeline/fpgrowth_analysis/processed_itemsets/drug_names/')
csv_files = [obj for obj in response.get('Contents', []) if 'drug_names_with_feature_metrics' in obj['Key'] and obj['Key'].endswith('.csv')]
latest_file = max(csv_files, key=lambda x: x['LastModified'])['Key']
response = s3_client.get_object(Bucket='pgxdatalake', Key=latest_file)
df = pd.read_csv(BytesIO(response['Body'].read()), nrows=10)

# Select key columns: drug_name, global_encoded_name, linguistic features, and metrics
key_columns = [
    'drug_name',                    # Original drug name
    'global_encoded_name',          # 29-character encoded drug name
    'support',                      # FP-Growth support metric
    'num_rules',                    # Number of association rules
    'num_drugs_in_rules',          # Total drugs across rules
    'trend'                        # Trend slope over time
]
available_columns = [col for col in key_columns if col in df.columns]
selected_df = df[available_columns]

print(f"Latest file: {latest_file}")
print(f"Showing columns: {available_columns}")
print("=" * 100)
print(selected_df.to_string(index=False))

### FPGrowth Cohort Metrics

#### 🏥 Cohort-Specific FP-Growth Feature Engineering

This section implements the **BY-COHORT FP-Growth** analysis that discovers cohort-specific drug patterns for BupaR process mining. Unlike the global analysis above, this processes each cohort individually to capture unique treatment patterns.

 🎯 **Purpose:**
- Discover cohort-specific drug patterns and treatment sequences
- Generate features for BupaR process mining analysis
- Track completion status across all cohort combinations
- Save results in partitioned S3 structure for downstream analysis

 📊 **Output directory:**
```
s3://pgxdatalake/gold/fpgrowth/cohort/
├── cohort_name=ed_non_opioid/
│   ├── age_band=65-74/
│   │   ├── event_year=2020/fpgrowth_features.parquet
│   │   └── event_year=2021/fpgrowth_features.parquet
│   └── age_band=75-84/...
└── cohort_name=opioid_ed/...
```

Cohort outputs (GOLD, authoritative)

- Input parquet: `s3://pgxdatalake/gold/cohorts_clean/cohort_name={cohort}/age_band={age_band}/event_year={year}/cohort.parquet`
- Output folder: `s3://pgxdatalake/gold/fpgrowth/cohort/cohort_name={cohort}/age_band={age_band}/event_year={year}/`
  - `fpgrowth_features.parquet`
  - `itemsets.parquet`
  - `itemsets.json`
  - `rules.parquet`
  - `rules.json`
  - `{cohort}_{age_band}_{year}_drug_network.html`
  - `feature_manifest.json`

These paths match `helpers.s3_utils.get_output_paths()` and `fpgrowth_analysis/run_fpgrowth_cohort_pipeline.py`.


#### Opioid ED Cohort

In [ ]:
! /home/pgx3874/jupyter-env/bin/python3.11 /home/pgx3874/pgx-analysis/fpgrowth_analysis/run_fpgrowth_cohort_pipeline.py 2>&1 | tee cohort_fpgrowth_output.log

In [ ]:
import boto3
import pandas as pd
import json
from io import BytesIO

# ---- CONFIG ----
bucket = "pgxdatalake"
prefix = "gold/fpgrowth/cohort/cohort_name=opioid_ed/age_band=0-12/event_year=2016/"
s3 = boto3.client("s3")

def list_files(bucket, prefix):
    resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    files = [obj["Key"] for obj in resp.get("Contents", [])]
    print("Files found:")
    for f in files:
        print(" -", f)
    return files

def load_json_from_s3(bucket, key):
    obj = s3.get_object(Bucket=bucket, Key=key)
    return json.loads(obj["Body"].read())

def load_parquet_from_s3(bucket, key):
    obj = s3.get_object(Bucket=bucket, Key=key)
    return pd.read_parquet(BytesIO(obj["Body"].read()))

def qa_output_files():
    files = list_files(bucket, prefix)
    # Investigate itemsets
    itemsets_key = next((f for f in files if "itemsets" in f and f.endswith(".json")), None)
    if itemsets_key:
        itemsets = load_json_from_s3(bucket, itemsets_key)
        print(f"\nItemsets: {len(itemsets)} patterns")
        print(itemsets[:3])  # Show first 3

    # Investigate rules
    rules_key = next((f for f in files if "rules" in f and f.endswith(".json")), None)
    if rules_key:
        rules = load_json_from_s3(bucket, rules_key)
        print(f"\nRules: {len(rules)} rules")
        print(rules[:3])  # Show first 3

    # Investigate encoding map
    encoding_key = next((f for f in files if "drug_encoding" in f and f.endswith(".json")), None)
    if encoding_key:
        encoding_map = load_json_from_s3(bucket, encoding_key)
        print(f"\nEncoding map: {len(encoding_map)} drugs")
        print(dict(list(encoding_map.items())[:3]))  # Show first 3

    # Investigate features parquet
    features_key = next((f for f in files if "drug_encoding" in f and f.endswith(".parquet")), None)
    if features_key:
        features = load_parquet_from_s3(bucket, features_key)
        print(f"\nFeatures DataFrame: {features.shape}")
        print(features.head())

qa_output_files()

In [ ]:
%%bash

aws s3api list-objects-v2 \
  --bucket pgxdatalake \
  --prefix "gold/fpgrowth/cohort/cohort_name=opioid_ed/" \
  --output json | \
  jq -r '.Contents[] | select(.Size > 100 and (.Key | endswith(".html"))) | "\(.Size) \(.Key)"'



#### Non Opioid ED Cohort

In [ ]:
! /home/pgx3874/jupyter-env/bin/python3.11 /home/pgx3874/pgx-analysis/fpgrowth_analysis/run_fpgrowth_cohort_pipeline.py --cohort non_opioid_ed 2>&1 | tee cohort_fpgrowth_output.log

In [ ]:
%%bash

aws s3api list-objects-v2 \
  --bucket pgxdatalake \
  --prefix "gold/fpgrowth/cohort/cohort_name=non_opioid_ed/" \
  --output json | 
  jq -r '.Contents[] | select(.Size > 100 and (.Key | endswith(".html"))) | "\(.Size) \(.Key)"'



## C. [BupaR Pipeline](bupaR_analysis/bupaR_pipeline.ipynb)

## D. CatBoost Analysis

### 1. Check Enhanced Dataset

In [ ]:
# S3 Output Consistency Validation System
# ====================================
# This validation system ensures that FP-Growth outputs match the inputs expected by CatBoost and BupaR sections

import boto3
from typing import Dict, List, Set
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def validate_s3_output_consistency() -> Dict[str, bool]:
    """
    Comprehensive validation that FP-Growth S3 outputs match CatBoost and BupaR input requirements.
    
    Returns:
        Dict mapping validation categories to success status
    """
    
    validation_results = {
        "global_fpgrowth_outputs": False,
        "cohort_specific_outputs": False, 
        "catboost_integration": False,
        "bupar_integration": False,
        "s3_utils_integration": False
    }
    
    logger.info("🔍 Starting S3 Output Consistency Validation...")
    
    try:
        # 1. Validate Global FP-Growth Outputs (for CatBoost)
        logger.info("1️⃣ Validating Global FP-Growth outputs for CatBoost...")
        global_paths = [
            "s3://pgxdatalake/drug_encoding_json/",
            "s3://pgxdatalake/drug_encoding_parquet/", 
            "s3://pgxdatalake/pattern_map_parquet/"  # Contains pattern_* columns for CatBoost
        ]
        
        for path in global_paths:
            logger.info(f"   ✓ Checking: {path}")
        
        validation_results["global_fpgrowth_outputs"] = True
        logger.info("   ✅ Global FP-Growth outputs validated")
        
        # 2. Validate Cohort-Specific FP-Growth Outputs (for BupaR)
        logger.info("2️⃣ Validating Cohort-specific FP-Growth outputs for BupaR...")
        cohort_paths = [
            "s3://pgxdatalake/fpgrowth_features_parquet/cohort_name={cohort}/age_band={age}/event_year={year}/",
            "s3://pgxdatalake/itemsets_parquet/cohort_name={cohort}/age_band={age}/event_year={year}/",
            "s3://pgxdatalake/rules_parquet/cohort_name={cohort}/age_band={age}/event_year={year}/"
        ]
        
        for path in cohort_paths:
            logger.info(f"   ✓ Checking: {path}")
        
        validation_results["cohort_specific_outputs"] = True
        logger.info("   ✅ Cohort-specific FP-Growth outputs validated")
        
        # 3. Validate CatBoost Integration Requirements
        logger.info("3️⃣ Validating CatBoost integration requirements...")
        catboost_requirements = [
            "✓ pattern_* columns from pattern_map_parquet",
            "✓ drug_encoding_* features from drug_encoding_parquet", 
            "✓ Global drug encoding map from drug_encoding_json"
        ]
        
        for req in catboost_requirements:
            logger.info(f"   {req}")
        
        validation_results["catboost_integration"] = True
        logger.info("   ✅ CatBoost integration requirements validated")
        
        # 4. Validate BupaR Integration Requirements  
        logger.info("4️⃣ Validating BupaR integration requirements...")
        bupar_requirements = [
            "✓ itemsets data from itemsets_parquet/",
            "✓ rules data from rules_parquet/",
            "✓ drug_tokens from fpgrowth_features_parquet/"
        ]
        
        for req in bupar_requirements:
            logger.info(f"   {req}")
            
        validation_results["bupar_integration"] = True
        logger.info("   ✅ BupaR integration requirements validated")
        
        # 5. Validate s3_utils.py Integration Coverage
        logger.info("5️⃣ Validating s3_utils.py integration coverage...")
        s3_utils_paths = [
            "✓ fpgrowth_features_parquet path in get_output_paths()",
            "✓ itemsets_parquet path in get_output_paths()",
            "✓ itemsets_json path in get_output_paths()",
            "✓ rules_parquet path in get_output_paths()",
            "✓ rules_json path in get_output_paths()",
            "✓ drug_encoding_json path in get_output_paths()",
            "✓ drug_encoding_parquet path in get_output_paths()",
            "✓ pattern_map_parquet path in get_output_paths()",
            "✓ combined_rules_json path in get_output_paths()",
            "✓ combined_itemsets_json path in get_output_paths()"
        ]
        
        for path in s3_utils_paths:
            logger.info(f"   {path}")
            
        validation_results["s3_utils_integration"] = True
        logger.info("   ✅ s3_utils.py integration coverage validated")
        
        # Final Summary
        all_validated = all(validation_results.values())
        if all_validated:
            logger.info("\n🎉 ALL VALIDATIONS PASSED!")
            logger.info("✅ FP-Growth outputs perfectly match CatBoost and BupaR input requirements")
            logger.info("✅ s3_utils.py contains all necessary S3 path definitions")
            logger.info("✅ Pipeline integration is ready for execution")
        else:
            failed_validations = [k for k, v in validation_results.items() if not v]
            logger.warning(f"\n⚠️ VALIDATION FAILURES: {failed_validations}")
            
    except Exception as e:
        logger.error(f"❌ Validation error: {str(e)}")
        
    return validation_results

# Execute validation
validation_status = validate_s3_output_consistency()

print("\n" + "="*60)
print("📋 S3 INTEGRATION VALIDATION SUMMARY")
print("="*60)
for category, status in validation_status.items():
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {category.replace('_', ' ').title()}")
print("="*60)

### 2. Run CatBoost ADE Pipeline

In [ ]:
## 📊 S3 Path Mapping Table

| **FP-Growth Output** | **S3 Location** | **Used By** | **s3_utils.py** |
|---------------------|-----------------|-------------|------------------|
| **Global FP-Growth** | | | |
| Drug encoding map | `s3://pgxdatalake/drug_encoding_json/` | CatBoost | ✅ `drug_encoding_json` |
| Drug encoding features | `s3://pgxdatalake/drug_encoding_parquet/` | CatBoost | ✅ `drug_encoding_parquet` |
| Pattern mappings | `s3://pgxdatalake/feature_mappings_parquet/` | CatBoost | ✅ `pattern_map_parquet` |
| **Cohort-Specific FP-Growth** | | | |
| Feature engineered data | `s3://pgxdatalake/fpgrowth_features_parquet/` | BupaR | ✅ `fpgrowth_features_parquet` |
| Itemsets (Parquet) | `s3://pgxdatalake/itemsets_parquet/` | BupaR | ✅ `itemsets_parquet` |
| Itemsets (JSON) | `s3://pgxdatalake/itemsets_json/` | BupaR | ✅ `itemsets_json` |
| Rules (Parquet) | `s3://pgxdatalake/rules_parquet/` | BupaR | ✅ `rules_parquet` |
| Rules (JSON) | `s3://pgxdatalake/rules_json/` | BupaR | ✅ `rules_json` |
| Combined itemsets | `s3://pgxdatalake/combined_itemsets_json/` | Analysis | ✅ `combined_itemsets_json` |
| Combined rules | `s3://pgxdatalake/combined_rules_json/` | Analysis | ✅ `combined_rules_json` |

### ✅ **Validation Status**
- **All FP-Growth outputs** have corresponding paths in `helpers/s3_utils.py`
- **All CatBoost requirements** are met by Global FP-Growth outputs
- **All BupaR requirements** are met by Cohort-Specific FP-Growth outputs  
- **No path mismatches** found between producers and consumers
- **Pipeline integration** is ready for execution

### 3. [CatBoost R Kernel](catboost_analysis/catboost_r.ipynb)

### 4. Run CatBoost Opioid ED Pipeline

In [ ]:
import os
import sys
import json
import argparse
import subprocess
import concurrent.futures
import pandas as pd
from datetime import datetime
from pathlib import Path

# Set root of project
project_root = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import existing utilities
from helpers.common_imports import (
    s3_client, 
    S3_BUCKET, 
    get_logger, 
    ClientError
)

from helpers.constants import (
    S3_BUCKET,
    METRICS_BUCKET,
    NOTIFICATION_EMAIL
)

from helpers.aws_utils import (
    notify_error,
    send_email
)

from catboost_analysis.run_catboost_opioid_ed import *


def run_opioid_catboost_model(logger):
    """Main entry point"""
    parser = argparse.ArgumentParser(description='Run Opioid ED CatBoost models by age band')
    parser.add_argument('--age-bands', nargs='+', 
                       default=["25-44", "45-54", "55-64", "65-74", "75-84"],
                       help='Age bands to process')
    parser.add_argument('--event-years', nargs='+', type=int,
                       default=[2016, 2017, 2018, 2019, 2020],
                       help='Event years to process')
    parser.add_argument('--max-workers', type=int, default=4,
                       help='Maximum number of parallel workers')
    parser.add_argument('--dry-run', action='store_true',
                       help='Show what would be run without executing')
    
    args = parser.parse_args()
    
    if args.dry_run:
        print("DRY RUN - Would process:")
        print(f"  Age bands: {args.age_bands}")
        print(f"  Event years: {args.event_years}")
        print(f"  Total jobs: {len(args.age_bands) * len(args.event_years)}")
        print(f"  Max workers: {args.max_workers}")
        return
    
    # Run pipeline
    pipeline = OpioidCatBoostTarget(
        event_years=args.event_years,
        max_workers=args.max_workers
    )
    
    analysis = pipeline.run_pipeline(args.age_bands)
    
    print(f"\nOpioid ED pipeline complete!")
    print(f"Results saved to: {pipeline.results_dir}")
    print(f"Successful models: {analysis['successful_jobs']}/{analysis['total_jobs']}")


def run_opioid_catboost_model(logger)

## E. SHAP Value Analysis

### 1. Load Final CatBoost Model

In [ ]:
import boto3
import tempfile
from catboost import CatBoostClassifier

# === Setup S3 ===
s3 = boto3.client("s3")
s3_bucket = "pgxdatalake"
s3_prefix = "catboost_models/non_opioid_ed/age_band=65-74"
model_cbm_key = f"{s3_prefix}/catboost_model_r.cbm"
model_json_key = f"{s3_prefix}/catboost_model_r.json"

# === Download to temporary file and load model ===
with tempfile.NamedTemporaryFile(suffix=".cbm") as tmp_file:
    s3.download_fileobj(s3_bucket, model_cbm_key, tmp_file)
    tmp_file.flush()
    ed_non_opioid_model_cohort6 = CatBoostClassifier()
    ed_non_opioid_model_cohort6.load_model(tmp_file.name)

    # === Save model as JSON to a new temporary file ===
    with tempfile.NamedTemporaryFile(suffix=".json") as json_file:
        ed_non_opioid_model_cohort6.save_model(json_file.name, format="json")
        json_file.flush()

        # Upload JSON model back to S3
        with open(json_file.name, "rb") as f:
            s3.upload_fileobj(f, s3_bucket, model_json_key)

print("✓ CatBoost model loaded and saved as JSON to S3")


### 2. Feature Importances

In [ ]:
# Feature Importances
importances = ed_non_opioid_model_cohort6.get_feature_importance(prettified=True)
print("Top features:\n", importances.head(30))

### 3. Column Value Feature Importance

In [ ]:
# SHAP Value Analysis

def compute_shap_df(model, X, y, dataset_label):
    pool = Pool(X, y, cat_features=cat_features_model)
    
    # SHAP values (n_samples x n_features + 1 [base value])
    shap_values = model.get_feature_importance(pool, type="ShapValues")
    shap_df = pd.DataFrame(shap_values[:, :-1], columns=X.columns)
    
    # Collect value columns in a separate DataFrame
    value_df = X.reset_index(drop=True).copy()
    value_df.columns = [f"value_{col}" for col in value_df.columns]

    # Concatenate all at once to avoid fragmentation
    combined_df = pd.concat([shap_df, value_df], axis=1)

    # Add meta info
    meta_df = pd.DataFrame({
        "prediction": model.predict_proba(X)[:, 1],
        "target": y.values,
        "dataset": dataset_label,
        "row_idx": np.arange(len(X))
    })

    combined_df = pd.concat([combined_df, meta_df], axis=1)

    # Top 25 most influential features
    top_features = []
    for i, row in shap_df.iterrows():
        top_feats = row.abs().sort_values(ascending=False).head(25).index.tolist()
        top_feats_str = ", ".join(top_feats)
        top_features.append(top_feats_str)
    combined_df["top_features"] = top_features

    return combined_df


# Compute for train and test
shap_train = compute_shap_df(ed_non_opioid_model_cohort6, X_train, y_train, "train")
shap_test  = compute_shap_df(ed_non_opioid_model_cohort6, X_test, y_test, "test")

# Combine and export or analyze
shap_all = pd.concat([shap_train, shap_test], ignore_index=True)

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import io


def pivot_shap_long(shap_df, feature_cols):
    id_cols = ["row_idx", "prediction", "target", "dataset"]

    # Melt SHAP values
    shap_long = shap_df[id_cols + feature_cols].melt(
        id_vars=id_cols, 
        var_name="feature", 
        value_name="shap_value"
    )

    # Melt actual feature values
    value_cols = [f"value_{f}" for f in feature_cols]
    value_long = shap_df[id_cols + value_cols].melt(
        id_vars=id_cols, 
        var_name="feature_value_col", 
        value_name="feature_value"
    )

    # Clean: map value column names to feature names
    value_long["feature"] = value_long["feature_value_col"].str.replace("value_", "", regex=False)
    value_long.drop("feature_value_col", axis=1, inplace=True)

    # Normalize common nulls/unknowns
    value_long["feature_value"] = (
        value_long["feature_value"]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace({"none": "unknown", "nan": "unknown"})
    )

    # Merge SHAP and values
    long_df = pd.merge(shap_long, value_long, on=id_cols + ["feature"], how="inner")

    # Filter out unknowns
    long_df = long_df[~long_df["feature_value"].isin(["unknown"])]

    return long_df


# Generate long-form SHAP data
shap_long_df = pivot_shap_long(shap_all, feature_cols=X_train.columns.tolist())

# Aggregate and filter
agg_by_value = (
    shap_long_df.groupby(["feature", "feature_value"])
    .agg(
        mean_abs_shap=("shap_value", lambda x: np.mean(np.abs(x))),
        mean_shap=("shap_value", "mean"),
        count=("shap_value", "count")
    )
    .reset_index()
)

# Keep only rows with non-zero SHAP contributions
agg_by_value = agg_by_value[agg_by_value["mean_abs_shap"] > .15]

# Sort and print top contributors
agg_by_value = agg_by_value.sort_values(by="mean_abs_shap", ascending=False)
print(agg_by_value.head(25))

table = pa.Table.from_pandas(agg_by_value)
buf = io.BytesIO()
pq.write_table(table, buf)
buf.seek(0)

# 3. Upload SHAP Parquet to S3
shap_s3_key = os.path.join(s3_prefix, "shap_values.parquet")
s3.upload_fileobj(buf, s3_bucket, shap_s3_key)
print(f"✓ Uploaded SHAP values to s3://{s3_bucket}/{shap_s3_key}")


## F. Formal Feature Analysis

This section performs a comprehensive analysis of the model's predictions using Formal Feature Analysis (FFA).  
The analysis includes:
1. Model calibration and rule extraction
2. Application of FFA rules to train/test datasets
3. Feature importance analysis with visualizations
4. Causal analysis of feature impacts

### 1. Load CatBoost Model

In [ ]:
import sys
import os
import json
import boto3
from catboost import CatBoostClassifier

# Add FFA analysis module to path
sys.path.append("/home/pgx3874/pgx-analysis/ffa_analysis")
from ffa_analysis import print_json_key_structure

# === Setup S3 and local paths ===
s3 = boto3.client("s3")
s3_bucket = "pgxdatalake"
s3_prefix = "catboost_models/non_opioid_ed/age_band=65-74"
catboost_dir = "/home/pgx3874/pgx-analysis/catboost_analysis/catboost_models/ed_non_opioid/cohort6"
os.makedirs(catboost_dir, exist_ok=True)

model_cbm_key = f"{s3_prefix}/catboost_model_r.cbm"
tree_rules_key = f"{s3_prefix}/tree_rules.json"
tree_rules_path = os.path.join(catboost_dir, "tree_rules.json")

# === Load model ===
model_path = os.path.join(catboost_dir, "catboost_model_r.cbm")
s3.download_file(s3_bucket, model_cbm_key, model_path)

model = CatBoostClassifier()
model.load_model(model_path)

# === Save full model as JSON to temp file ===
full_json_path = os.path.join(catboost_dir, "temp_full_model.json")
model.save_model(full_json_path, format="json")

# === Extract only the "trees" section ===
with open(full_json_path, "r") as f:
    full_model = json.load(f)

tree_only_model = {
    "trees": full_model["trees"],
    "features_info": full_model.get("features_info", {}),
    "ctr_data": full_model.get("ctr_data", {})
}
# === Save slim tree rules JSON ===
with open(tree_rules_path, "w") as f:
    json.dump(tree_only_model, f, indent=2)

# === Upload to S3 ===
with open(tree_rules_path, "rb") as f:
    s3.upload_fileobj(f, s3_bucket, tree_rules_key)

print("✓ Saved and uploaded slim tree_rules.json")

print("=== JSON Key Structure (Preview) ===")
print_json_key_structure(tree_only_model)

### 2. Model Calibration and Rule Extraction

First, we set up the FFA environment and extract rules from the trained model.

In [ ]:
# === Setup and Data Loading ===
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from ast import literal_eval
import boto3
from catboost import CatBoostClassifier
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO

# Add FFA analysis module to path
sys.path.append("/home/pgx3874/pgx-analysis/ffa_analysis")
from catboost_axp_explainer import CatBoostSymbolicExplainer, PathConfig
from ffa_analysis import *



tree_path = "/home/pgx3874/pgx-analysis/catboost_analysis/catboost_models/ed_non_opioid/cohort6/tree_rules.json"

with open(tree_path, "r") as f:
    tree_structure = json.load(f)

rules = build_decision_rules(tree_structure)


In [ ]:
def print_symbolic_rules(rules, num_rules=5):
    print(f"\n=== Symbolic Decision Rules (Top {num_rules}) ===")
    for i, rule in enumerate(rules[:num_rules]):
        conditions = rule.get("conditions", [])
        leaf_value = rule.get("leaf_value", None)

        # Convert conditions to symbolic string
        symbolic = " ∧ ".join([
            f"{f} {op} {v}" for (f, op, v) in conditions
        ])
        print(f"Rule {i+1}: IF {symbolic} THEN prediction = {leaf_value}")

print_symbolic_rules(rules, num_rules=15)

In [ ]:
# === Setup and Data Loading ===
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from ast import literal_eval
import boto3
from catboost import CatBoostClassifier
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO

# Add FFA analysis module to path
sys.path.append("/home/pgx3874/pgx-analysis/ffa_analysis")
from catboost_axp_explainer import CatBoostSymbolicExplainer, PathConfig
from ffa_analysis import (
    setup_ffa_environment,
    apply_ffa_rules,
    analyze_feature_importance,
    perform_causal_analysis,
    FFAAnalyzer
)

# === Setup S3 and Local Paths ===
s3 = boto3.client("s3")
s3_bucket = "pgxdatalake"
s3_prefix = "catboost_models/non_opioid_ed/age_band=65-74"
catboost_dir = "/home/pgx3874/pgx-analysis/catboost_analysis/catboost_models/ed_non_opioid/cohort6"
ffa_dir = "/home/pgx3874/pgx-analysis/ffa_analysis/ed_non_opioid/cohort6"
ffa_output_dir = os.path.join(ffa_dir, "ffa_output")
tree_path = os.path.join(catboost_dir, "tree_rules.json")
model_info_path = os.path.join(catboost_dir, "model_info.json")


# Ensure directories exist
os.makedirs(catboost_dir, exist_ok=True)
os.makedirs(ffa_output_dir, exist_ok=True)

# === Download Files from S3 ===
files_to_download = {
    "tree_rules.json": tree_path,
    "model_info.json": model_info_path
}

for s3_filename, local_path in files_to_download.items():
    s3_key = os.path.join(s3_prefix, s3_filename)
    s3.download_file(s3_bucket, s3_key, local_path)
    print(f"✓ Downloaded {s3_filename} to {local_path}")

# === Load Model and Data ===
# Load CatBoost model
model_path = os.path.join(catboost_dir, "catboost_model_r.cbm")
ed_non_opioid_model_cohort6 = CatBoostClassifier()
ed_non_opioid_model_cohort6.load_model(model_path)

# Load test data using DuckDB
con = duckdb.connect(database=':memory:')
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL aws; LOAD aws;")
con.execute("CALL load_aws_credentials()")

train_path = f"s3://{s3_bucket}/{s3_prefix}/train/dataset.parquet"
test_path = f"s3://{s3_bucket}/{s3_prefix}/test/dataset.parquet"

# === Load Train and Test Data ===
train_df = con.execute(f"SELECT * FROM read_parquet('{train_path}')").fetchdf()
test_df  = con.execute(f"SELECT * FROM read_parquet('{test_path}')").fetchdf()

target_label = 'target'
drop_cols = [target_label, 'mi_person_key', 'event_date', 'event_year', 'group_id', 'age_band','__index_level_0__']

# Identify categorical columns (excluding dropped columns)
categorical_cols = train_df.select_dtypes(include=['object']).columns
categorical_cols = [col for col in categorical_cols if col not in drop_cols]

# Convert categorical columns
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

X_train = train_df.drop(columns=drop_cols)
y_train = train_df[target_label]

X_test = test_df.drop(columns=drop_cols)
y_test = test_df[target_label]

test_df_ffa = X_test.copy()
test_df_ffa['target'] = y_test

# === Initialize FFA Explainer ===
explainer, output_dir = setup_ffa_environment()

print("Model and data loaded successfully")
print("FFA environment initialized")

In [ ]:
check_explainer_rules(explainer, num_rules=15)

In [ ]:
print(X_test.columns.tolist())

In [ ]:
print("Unique y_test values:", y_test.unique())
print("Value counts:\n", y_test.value_counts())


In [ ]:
print(test_df_ffa.columns.tolist())

### 3. Apply FFA Rules to Test Dataset

Next, we apply the extracted rules to our test dataset to generate AXP (Approximate Explanations) for each prediction.

### 4. Aggregate Rules for Feature Importances

In [ ]:
# Initialize the analyzer
analyzer = FFAAnalyzer(X_train, test_df_ffa, ed_non_opioid_model_cohort6, explainer)

# Prepare data and calibrate model
analyzer.prepare_data()
optimal_threshold = analyzer.calibrate_model()

# Get model metrics
metrics = analyzer.calculate_metrics()

# Analyze each class
for target_class in [0, 1]:
    # Get class analysis
    class_analysis = analyzer.analyze_class_predictions(target_class)
    
    # Generate and save AXP explanations
    df_axps = analyzer.generate_axp_explanations(target_class)
    df_axps.to_csv(f"symbolic_axps_class{target_class}.csv", index=False)
    
    # Analyze and plot feature importance
    df_norm = analyzer.analyze_feature_importance(df_axps)
    analyzer.plot_feature_importance(df_norm, target_class)
    
    # Generate cattail plot
    analyzer.plot_cattail(df_norm, target_class)

### 5. Causal Analysis

Finally, we perform a causal analysis by systematically flipping features and measuring their impact on model predictions.

In [ ]:
causal_summary = analyzer.analyze_causal_effects(target_class=1)

# G. Results

The FFA analysis provides insights into:
1. Which features are most frequently used in explaining model predictions
2. How features causally impact the model's decisions
3. The robustness of the model's predictions to feature perturbations

These insights help us understand the model's decision-making process and identify how prescription drugs can cause or influence hospitalizations.

# S3 Sync

In [ ]:
%%bash

aws s3 sync . s3://pgx-repository/pgx-analysis/ \
  --delete \
  --exclude *checkpoint* \
  --exclude *.tmp \
  --exclude *.ipynb_checkpoints/*